In [23]:
#단층 신경망 구현

import tensorflow as tf
import numpy as np #수치해석용 라이브러리, 행력조작, 연사에 필수

#[털, 날개]
x_data = np.array([[0,0],[1,0],[1,1],[0,0],[0,0],[0,1]])


#레이블 데이터(one-hot encodeing)
#기타 = [1,0,0]
#포유류 = [0,1,0]
#조류 = [0,0,1]

#레이블 데이터 구성
y_data = np.array([
    [1,0,0], # 기타
    [0,1,0], # 포유류
    [0,0,1], # 조류
    [1,0,0],
    [1,0,0],
    [0,0,1]
])

#[털,날개] -> [기타,포유류,조류]
#[0,0] -> [1,0,0] #기타
#[1,0] -> [0,1,0] #포유류
#[1,1] -> [0,0,1] #조류
#[0,0] -> [1,0,0] #기타
#[0,0] -> [1,0,0] #기타
#[0,1] -> [0,0,1] #기타


#신경망 모델 구현
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(tf.random_uniform([2,3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

#가중치를 곱하고 편향을 더한 결과를 활성화 함수인 ReLU에 적용
L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L)

#softmax()는 배열 내의 결괏괎들을 전체 합이 1이 되도록 만들어준다 즉 확률로 환산시켜줌
model = tf.nn.softmax(L)

#손실함수 작성(교차 엔트로피 함수 사용, 교차 엔트로피 값 : 에측값과 실제값 사이의 확률 분포 차이를 계산한 값)
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(model), 1))


#경사하강법 최적화
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

        
#텐서플로의 세션 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

#학습
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    #학습도중 10번에 한번 씩 손실값 출력
    if(step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
        
        
#결과확인 코드
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값 :', sess.run(prediction, feed_dict={X: x_data}))
print('실제값 :', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))


10 1.1532916
20 1.1489494
30 1.1449142
40 1.1435634
50 1.1422349
60 1.1409285
70 1.1396437
80 1.1383799
90 1.137137
100 1.1359147
예측값 : [0 0 0 0 0 0]
실제값 : [0 1 2 0 0 2]
정확도: 50.00


In [28]:
#다층 신경망

import tensorflow as tf
import numpy as np #수치해석용 라이브러리, 행력조작, 연사에 필수

#[털, 날개]
x_data = np.array([[0,0],[1,0],[1,1],[0,0],[0,0],[0,1]])


#레이블 데이터(one-hot encodeing)
#기타 = [1,0,0]
#포유류 = [0,1,0]
#조류 = [0,0,1]

#레이블 데이터 구성
y_data = np.array([
    [1,0,0], # 기타
    [0,1,0], # 포유류
    [0,0,1], # 조류
    [1,0,0],
    [1,0,0],
    [0,0,1]
])

#[털,날개] -> [기타,포유류,조류]
#[0,0] -> [1,0,0] #기타
#[1,0] -> [0,1,0] #포유류
#[1,1] -> [0,0,1] #조류
#[0,0] -> [1,0,0] #기타
#[0,0] -> [1,0,0] #기타
#[0,1] -> [0,0,1] #기타


#신경망 모델 구현(단층신경망과 다른점 가중치와 편향치가 몇개 더 있음. 즉, 은닉층이 생김.)
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.))
W2 = tf.Variable(tf.random_uniform([10,3], -1., 1.))

b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))


#가중치를 곱하고 편향을 더한 결과를 활성화 함수인 ReLU에 적용
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

#softmax()는 배열 내의 결괏괎들을 전체 합이 1이 되도록 만들어준다 즉 확률로 환산시켜줌
model = tf.add(tf.matmul(L1, W2), b2)

#손실함수 작성(교차 엔트로피 함수 사용, 교차 엔트로피 값 : 에측값과 실제값 사이의 확률 분포 차이를 계산한 값)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))


#AdamOptimizer 최적화
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

        
#텐서플로의 세션 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

#학습
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    #학습도중 10번에 한번 씩 손실값 출력
    if(step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
        
        
#결과확인 코드
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값 :', sess.run(prediction, feed_dict={X: x_data}))
print('실제값 :', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))


10 1.0459524
20 0.8989875
30 0.76354474
40 0.6382885
50 0.5139943
60 0.39800394
70 0.29253516
80 0.20499003
90 0.14181708
100 0.10040649
예측값 : [0 1 2 0 0 2]
실제값 : [0 1 2 0 0 2]
정확도: 100.00
